In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 15716499
paper_name = 'deutschbauer_giaever_2005' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/OrfGeneData.xlsx', sheet_name='OrfGeneData')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 5922 x 18


In [7]:
original_data.head()

,orf,gene,go_process,go_function,go_component,essential,het_ypd_A,het_ypd_B,HET_AV,slow_ypd_het,hom_ypd_C,hom_ypd_D,HOM_AV,slow_ypd_hom,het_mm_B,slow_mm_het,hom_mm_D,slow_mm_hom
0,YAL001C,TFC3,transcription initiation from Pol III promoter,RNA polymerase III transcription factor activity,transcription factor TFIIIC complex,YES,1.006,1.007,1.006,no,ND,ND,ND,Essential,1.007,no,ND,Essential
1,YAL002W,VPS8,late endosome to vacuole transport,molecular_function unknown,membrane fraction,NO,0.953,0.928,0.941,no,0.976,0.991,0.983,no,1.013,no,0.837,YES
2,YAL003W,EFB1,translational elongation,translation elongation factor activity,ribosome,YES,1.019,1.01,1.014,no,ND,ND,ND,Essential,0.987,no,ND,Essential
3,YAL004W,YAL004W,NaN,NaN,NaN,NO,1.019,0.998,1.009,no,1,1.011,1.005,no,0.999,no,0.999,NO
4,YAL005C,ssa1,protein folding*,chaperone activity*,cytoplasm*,NO,0.97,0.931,0.95,no,1.006,1.009,1.008,no,0.933,no,1.013,NO


In [8]:
original_data['orf'] = original_data['orf'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [orf, gene, go_process, go_function, go_component, essential, het_ypd_A, het_ypd_B, HET_AV, slow_ypd_het, hom_ypd_C, hom_ypd_D, HOM_AV, slow_ypd_hom, het_mm_B, slow_mm_het, hom_mm_D, slow_mm_hom]
Index: []


In [12]:
original_data.set_index('orf', inplace=True)

In [13]:
original_data = original_data[['HET_AV','HOM_AV','het_mm_B','hom_mm_D']].apply(pd.to_numeric, axis=1, errors='coerce')

In [14]:
original_data = original_data.groupby(original_data.index).mean()

In [15]:
original_data.shape

(5890, 4)

# Prepare the final dataset

In [16]:
data = original_data.copy()

In [17]:
dataset_ids = [521, 5257, 522, 5258]
datasets = datasets.reindex(index=dataset_ids)

In [18]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [19]:
data.head()

dataset_id,521,5257,522,5258
data_type,value,value,value,value
orf,,,,
YAL001C,1.006,NaN,1.007,NaN
YAL002W,0.941,0.983,1.013,0.837
YAL003W,1.014,NaN,0.987,NaN
YAL004W,1.009,1.005,0.999,0.999
YAL005C,0.950,1.008,0.933,1.013


## Subset to the genes currently in SGD

In [20]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 24


In [21]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,521,5257,522,5258
,data_type,value,value,value,value
gene_id,orf,,,,
1,YAL001C,1.006,NaN,1.007,NaN
2,YAL002W,0.941,0.983,1.013,0.837
3,YAL003W,1.014,NaN,0.987,NaN
1863,YAL004W,1.009,1.005,0.999,0.999
4,YAL005C,0.950,1.008,0.933,1.013


# Normalize

In [22]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [23]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [24]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,521,5257,522,5258,521,5257,522,5258
,data_type,value,value,value,value,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,
1,YAL001C,1.006,NaN,1.007,NaN,0.184571,NaN,0.205387,NaN
2,YAL002W,0.941,0.983,1.013,0.837,-2.853061,-0.435290,0.326418,-2.043128
3,YAL003W,1.014,NaN,0.987,NaN,0.558433,NaN,-0.198052,NaN
1863,YAL004W,1.009,1.005,0.999,0.999,0.324769,-0.092782,0.044011,-0.170796
4,YAL005C,0.950,1.008,0.933,1.013,-2.432466,-0.046077,-1.287336,-0.008989


# Print out

In [25]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [26]:
from IO.save_data_to_db3 import *

In [27]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/4 [00:00<?, ?it/s]

Deleting all datasets for PMID 15716499...
Inserting the new data...


100%|██████████| 4/4 [00:36<00:00,  9.10s/it]

Updating the data_modified_on field...
